## Bias correction testing ##

Now that we have established that there are biases in the HadGEM3-GC31-MM ensemble when compared to ERA5 on monthly timescales, for temperature, solar irradiance, and 10m wind speeds, we can now go about testing the bias correction which is applied to the daily data. Utilizing methods from Dawkins et al. (2021) and Luo et al., (2018).

* Dawkins et al. (2021) - https://catalogue.ceda.ac.uk/uuid/7beeed0bc7fa41feb10be22ee9d10f00?jump=related-docs-anchor

* Luo et al. (2018) - https://www.mdpi.com/2073-4441/10/8/1046

In [ ]:
import os
import sys
import glob
import importlib
from tqdm import tqdm

# Import relevant libraries
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

# # Optional dask stuff
# import dask_gateway
# import pathlib

In [ ]:
# Import the functsions
sys.path.append("/home/users/benhutch/unseen_functions/")

# import the bias functions
import bias_functions as bf

In [ ]:
# import the dictionaries
sys.path.append("/home/users/benhutch/unseen_functions/")

# import the dictionaries
import unseen_dictionaries as udicts

In [ ]:
# Set up the arguments
# --------------------
model = "HadGEM3-GC31-MM"
variable = "tas"
obs_variable = "t2m"
lead_time = 1 # first start november to end october period
experiment = "dcppA-hindcast"
freq = "day"
engine = "netcdf4"
parallel = False

# Init years for testing
init_years = [1960]

# Test file for daily data
test_file = "/badc/cmip6/data/CMIP6/DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1960-r1i1p1f2/day/tas/gn/files/d20200417/tas_day_HadGEM3-GC31-MM_dcppA-hindcast_s1960-r1i1p1f2_gn_19601101-19601230.nc"

# Daily obs fpath - single var - t2m
obs_fpath = "/home/users/benhutch/ERA5/ERA5_t2m_daily_1950_2020.nc"

In [ ]:
%%time

# Load the dcpp daily data
ds = bf.load_dcpp_data_lead(
    model=model,
    variable=variable,
    lead_time=lead_time,
    init_years=init_years,
    experiment=experiment,
    frequency=freq,
    engine=engine,
    parallel=parallel,
)

In [ ]:
%%time

# Regrid the data
ds = bf.regrid_ds(
    ds=ds,
    variable=variable,
)

In [ ]:
%%time

# Select the gridbox
ds = bf.select_gridbox(
    ds=ds,
    grid=dicts.eu_grid,
    calc_mean=False,
)

In [ ]:
%%time

# Load th test file
test_ds = xr.open_dataset(test_file)

# Load the daily observations
obs = bf.load_and_rg_obs(
    model_ds=test_ds,
    obs_variable=obs_variable,
    obs_path=obs_fpath,
    init_years=init_years,
    lead_time=lead_time,
    rg_algo="bilinear",
    grid_bounds=[-180.0, 180.0, -90.0, 90.0],
    periodic=True,
    parallel=False,
)